# Importing libraries


In [4]:
import pytesseract #for OCR
import pdfplumber # to extract text from pdf
from pdf2image import convert_from_path #to extract text from image based pdfs
import sqlite3 # for database
from PIL import Image #to manipulate images
import re #regular exp

# Extracting text from Image based Pdfs (OCR)

The text is extracted from the image based pdfs with the help of Poppler and Pytessaract libraries and is stored in the text_data variable. To run this code snippet, poppler has to be locally installed in your computer and the variable pop has to be updated with its path.

In [121]:
pop= r"D:\downloads\Release-24.08.0-0\poppler-24.08.0\Library\bin" 
pdf = r"bank_statement (3).pdf"
images = convert_from_path(pdf_path = pdf, poppler_path = pop)
text_data = ""
for img in images:
    text_data += pytesseract.image_to_string(img) + "\n"
text_data

'Bank Statement\n\nBank Name: XYZ Bank\nAccount Holder: John Doe\nAccount Number: 123456789\nStatement Date: February 2024\n\nDescription Amount\n\nGym Membership\n\n-$50.75\n$3000.00\n-$120.50\n\n01-02-24\n01-05-24\n\n01-10-24\n\n01-12-24 -$200.00\n\n01-18-24\n01-20-24\n01-22-24\n01-25-24\n\n\n'

# Extracting text from PDFs

This code uses pdfplumber library to extract the text from the pdf and stores it in the text_data variable. The user has to have the idea of what kind of pdf they are giving as an input and should only run either of the code snippet to avoid the over writing of the text_data variable.

In [142]:
text_data=""
with pdfplumber.open("bank_statement (3).pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            text_data += text + "\n"
            
text_data

'Bank Statement\nBank Name: XYZ Bank\nAccount Holder: John Doe\nAccount Number: 123456789\nStatement Date: February 2024\nDate Description Amount\n01-02-24 Grocery Store -$50.75\n01-05-24 Salary Deposit $3000.00\n01-07-24 Utility Bill -$120.50\n01-10-24 Online Purchase -$35.99\n01-12-24 ATM Withdrawal -$200.00\n01-15-24 Gas Station -$45.20\n01-18-24 Restaurant -$80.00\n01-20-24 Electric Bill -$90.50\n01-22-24 Internet Bill -$60.00\n01-25-24 Gym Membership -$40.00\n'

# Extracting Transaction details 

The transaction details are extracted from the text_data variable, with regular expressions we extract the transaction information like date, description and the transacted amount from the text_data.
Note: since this is only a prototype, we made a bank statement by ourselves and build the extraction code acccordingly which works for this template, this code will not work with bank statements of other formats.

In [145]:
transactions = []
transaction_pattern = r"(\d{2}-\d{2}-\d{2})\s+(.+?)\s+\$(-?\d+\.\d{2})"
matches = re.findall(transaction_pattern, text_data)

for date, description, amount in matches:
    try:
           amount = float(amount)
           transactions.append((date, description.strip(), amount))
    except ValueError:
           continue  

transactions

[('01-05-24', 'Salary Deposit', 3000.0)]

In [173]:

transactions = []
lines = text_data.split('\n')
for line in lines:
    if 'Date' in line or 'Bank' in line or 'Account' in line or 'Statement' in line or not line.strip():
        continue
    try:
        parts = line.split()
        if len(parts) >= 3:
            date = parts[0]
            amount = parts[-1].replace('$', '').replace('-$', '-')
            description = ' '.join(parts[1:-1])
            amount = float(amount)
            transactions.append((date, description, amount))
    except:
        continue
print("Extracted Transactions:")
for transaction in transactions:
    print(transaction)

Extracted Transactions:
('01-02-24', 'Grocery Store', -50.75)
('01-05-24', 'Salary Deposit', 3000.0)
('01-07-24', 'Utility Bill', -120.5)
('01-10-24', 'Online Purchase', -35.99)
('01-12-24', 'ATM Withdrawal', -200.0)
('01-15-24', 'Gas Station', -45.2)
('01-18-24', 'Restaurant', -80.0)
('01-20-24', 'Electric Bill', -90.5)
('01-22-24', 'Internet Bill', -60.0)
('01-25-24', 'Gym Membership', -40.0)


# Setting up the Database

We are using Sqlite3, a python inbuilt database, this code snippet initialises the database and creates a table with account number of the customer as its name(if doesnt exists) with date, description and amount as its columns.

In [87]:
match = re.search(r'Account Number:\s*(\d+)', text_data)
account_number= match.group(1) 
conn = sqlite3.connect("bank_statements.db")
cursor = conn.cursor()
query = f'''CREATE TABLE IF NOT EXISTS account_{account_number} (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    date TEXT, 
                    description TEXT, 
                    amount REAL
                )'''
cursor.execute(query)
conn.commit()
conn.close()

# Storing Transactions in Database

This code snippet stores the transaction in the database.

In [94]:
conn = sqlite3.connect("bank_statements.db")
cursor = conn.cursor()

query = f"INSERT INTO account_{account_number} (date, description, amount) VALUES (?, ?, ?)"
cursor.executemany(query, transactions)
    
conn.commit()
conn.close()
print(f" Transactions stored in table: account_{account_number}")

 Transactions stored in table: account_123456789


# Fetch the transactions from Database


This code snippet fetches the transaction details of the customer with the help of their account number

In [ ]:

conn = sqlite3.connect("bank_statements.db")
cursor = conn.cursor()

table_name = f"account_{account_number}" 

try:
    cursor.execute(f"SELECT date, description, amount FROM {table_name}")
    transactions = cursor.fetchall() 

    if not transactions:
        print(f" No transactions found for account {account_number}.")
    else:
        print(f"Transactions for Account {account_number}:")
        for transaction in transactions:
            print(transaction)

except sqlite3.Error as e:
    print(f"Database error: {e}")

finally:
    conn.close()
